## TF-IDF를 이용한 핵심어 추출
- 형태소 추출하기
    - 한글 텍스트로 전처리
    - 형태소 단위로 추출
- 분류 모델의 학습 데이터로 변환
    - corpus index 생성
    - TF-IDF로 변환

In [2]:
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

In [28]:
import re
from konlpy.tag import Okt

# 한글만 추출
def text_cleaning(text):
    hangul = re.compile('[^ ㄱ-ㅣ가-힣]+')
    result = hangul.sub('', text)
    return result

# 형태소 단위로 추출
def get_pos(x):
    tagger = Okt()
    pos = tagger.pos(x)
    pos = ['{}/{}'.format(word, tag) for word, tag in pos]
    return pos


In [29]:

df = pd.read_csv("review_data.csv")

# 한글만 추출
df['ko_text'] = df['review'].apply(lambda x: text_cleaning(str(x)))
del df['review']
df = df[df['ko_text'].str.len() > 0]

# 형태소 단위로 추출
result = get_pos(df['ko_text'].values[0])
print(result)

['맛있었어요/Adjective', '콩나물/Noun', '엄청/Adverb', '좋아해서/Adjective', '많이/Adverb', '먹었는데도/Verb', '많이/Adverb', '남았어요/Verb', '조금/Noun', '아까웠/Adjective', '슴/Noun', 'ㅠㅠ/KoreanParticle', '남은건/Verb', '포장/Noun', '도/Josa', '되서/Verb', '너무/Adverb', '좋았습니다/Adjective', '다음/Noun', '엔/Josa', '불고기/Noun', '먹으러/Verb', '가고싶어요/Verb', '너무/Adverb', '맛있게/Adjective', '잘/Verb', '먹었/Verb', '습/Noun', '니당/Verb']


In [34]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

# 형태소를 벡터 형태의 학습 데이터셋으로 변환
index_vectorizer = CountVectorizer(tokenizer = lambda x: get_pos(x))
X = index_vectorizer.fit_transform(df['ko_text'].to_list())
print(X.shape)
print(str(index_vectorizer.vocabulary_)[:100]+"..")
print(df['ko_text'].values[0])
print(X[0])

# TF-IDF로 변환
# 형태소를 벡터 형태의 학습데이터셋으로 변환
tfidf_vectorizer = TfidfTransformer()
X = tfidf_vectorizer.fit_transform(X)
print(X.shape)
print(X[0])

(37, 650)
{'맛있었어요/Adjective': 220, '콩나물/Noun': 581, '엄청/Adverb': 409, '좋아해서/Adjective': 524, '많이/Adverb': 206,..
맛있었어요콩나물 엄청 좋아해서 많이 먹었는데도 많이 남았어요 조금 아까웠슴 ㅠㅠ남은건 포장도 되서 너무 좋았습니다다음엔 불고기 먹으러 가고싶어요 너무 맛있게 잘 먹었습니당
  (0, 220)	1
  (0, 581)	1
  (0, 409)	1
  (0, 524)	1
  (0, 206)	2
  (0, 241)	1
  (0, 105)	1
  (0, 512)	1
  (0, 367)	1
  (0, 343)	1
  (0, 3)	1
  (0, 106)	1
  (0, 600)	1
  (0, 156)	1
  (0, 161)	1
  (0, 115)	2
  (0, 525)	1
  (0, 134)	1
  (0, 423)	1
  (0, 300)	1
  (0, 242)	1
  (0, 8)	1
  (0, 212)	1
  (0, 491)	1
  (0, 239)	1
  (0, 344)	1
  (0, 126)	1
(37, 650)
  (0, 600)	0.1554287617162363
  (0, 581)	0.18856482412924638
  (0, 525)	0.18856482412924638
  (0, 524)	0.18856482412924638
  (0, 512)	0.18856482412924638
  (0, 491)	0.16918147019607968
  (0, 423)	0.18856482412924638
  (0, 409)	0.16918147019607968
  (0, 367)	0.18856482412924638
  (0, 344)	0.18856482412924638
  (0, 343)	0.18856482412924638
  (0, 300)	0.18856482412924638
  (0, 242)	0.18856482412924638
  (0, 241)	0.18856482412924638
 

## 긍정 부정 리뷰 분류하기
- 분류 모델링
    - 데이터셋 분리
    - 로지스틱 회귀